# MNGEO API: CKAN

The [CKAN API](https://docs.ckan.org/en/ckan-2.1.5/api.html 'CKAN API') is used by Minnesota Geospatial Commons. This API can be used to do the following:
* Get JSON-formatted lists of a site’s datasets, groups or other CKAN objects
* Get a full JSON representation of a dataset, resource or other object
* Search for packages or resources matching a query
* Create, update and delete datasets, resources and other objects

To call the CKAN API, post a JSON dictionary in an HTTP POST request to one of CKAN’s API URLs. The parameters for the API function should be given in a JSON dictionary. CKAN will also return its response in a JSON dictionary.
The API aims to always return 200 OK as the status code of its HTTP response, whether there were errors with the request or not. Some API functions require authorization.

Since using this API deals with multiple JSON files, the json module should be imported.
The module zipfile is useful because the output will be shapefiles which should be zipped and saved to your computer. Arcpy is used in editing the output shapefiles, putting them into a geodatabase, and in showing what files are in the geodatabase. I used pandas to preview the data tables of the shapefiles.

In [106]:
import requests
import json
import pprint
import zipfile
import arcpy
import pandas as pd
import os

## Requesting Datasets from CKAN

In [107]:
# CKAN uses packages, groups and tags
# Setting up variables to hold each HTTP URL
packages = "https://gisdata.mn.gov/api/3/action/package_list"
groups = "https://gisdata.mn.gov/api/3/action/group_list"
tags = "https://gisdata.mn.gov/api/3/action/tag_list"

In [108]:
# Creating a request for the tags
response = requests.get(tags, auth = ('user', 'pass'), verify = False) #Test to see if tags can be retrieved

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\urllib3\connectionpool.py:1020: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gisdata.mn.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


In [109]:
# printing out the kind of data within the content of the response
type(response.content)

<class 'bytes'>

In [110]:
# Parsing a valid JSON string and converting it into a Python Dictionary
# Setting the dictionary to a variable 'tags_json'
tags_json = json.loads(response.content)

print(tags_json) #Messy example

{'help': 'https://gisdata.mn.gov/api/3/action/help_show?name=tag_list', 'success': True, 'result': ['100k index', '103e', '15 minute', '1994', '1-meter orthophoto', '1 water supply planning working groups', '2000 census', '2002', '2010', '2010 census', '2012', '2015', '2016', '2020', '2020 census', '2020 musa', '2030', '2030 framework', '2030 framework planning areas', '2030 musa', '2040', '2040 musa', '2050', '24k index', '24k streams', '250k index', '2 primary drinking water supply sources', '303d', '3 municipal wells per community in 1980 and 2040', '4 average daily municipal demand in 1980 and 2040', '5 2015 master water supply plan community profiles', '62500', '911', '9-1-1', '9-1-1 address validation', 'aadt', 'abandoned sewer interceptors', 'abrt', 'ac', 'aca', 'access', 'access trails', 'acp', 'acp50', 'acquisition', 'acquisitions', 'active living', 'activities', 'activity', 'activity center', 'activity level', 'addin', 'address', 'addresses', 'addressing', 'address points', '

In [111]:
pprint.pprint(tags_json) #Pretty example

{'help': 'https://gisdata.mn.gov/api/3/action/help_show?name=tag_list',
 'result': ['100k index',
            '103e',
            '15 minute',
            '1994',
            '1-meter orthophoto',
            '1 water supply planning working groups',
            '2000 census',
            '2002',
            '2010',
            '2010 census',
            '2012',
            '2015',
            '2016',
            '2020',
            '2020 census',
            '2020 musa',
            '2030',
            '2030 framework',
            '2030 framework planning areas',
            '2030 musa',
            '2040',
            '2040 musa',
            '2050',
            '24k index',
            '24k streams',
            '250k index',
            '2 primary drinking water supply sources',
            '303d',
            '3 municipal wells per community in 1980 and 2040',
            '4 average daily municipal demand in 1980 and 2040',
            '5 2015 master water supply plan community p

            'education',
            'educational program',
            'effluent',
            'election',
            'election district',
            'election results',
            'elections',
            'electric',
            'electrical utilities',
            'electrical utility',
            'electricity',
            'electric service areas',
            'electric utilities',
            'electric utility service area',
            'elementary school',
            'elemetary',
            'elev',
            'elevation',
            'elevation topography',
            'e line',
            'elink',
            'emerald ash borer',
            'emergency',
            'emergency medical service',
            'emergency medical service agency',
            'emergency response',
            'emergency response area',
            'emergency service agency',
            'emergency service number',
            'emergency service response agency',
            'emergency services',

            'metropolitan parks and open space commission',
            'metropolitan urban service area',
            'metro transit',
            'mgc',
            'mgs',
            'microwave',
            'microwave towers',
            'middle school',
            'migration',
            'migratory waterfowl',
            'mile',
            'mile marker',
            'mile measures',
            'minarchive',
            'mine locations',
            'mineral endowment',
            'mineral exploration',
            'mineral exploration documents',
            'mineral leases',
            'mineral potential',
            'mineral resources',
            'minerals',
            'mink',
            'minnesota',
            'minnesota department of health',
            'minnesota geographic metadata guidelines mgmg',
            'minnesota pollution control agency',
            'minnesota state highway',
            'minnesota state patrol',
            'minor civil division bo

            'served area',
            'service area',
            'sessions',
            'sewage',
            'sewer',
            'sewerage',
            'sewer interceptors',
            'sewers',
            'sewer service',
            'sewershed',
            'sewer systems',
            'sfha flood insurance rate map',
            'sgcn',
            'shaded relief',
            'shallow lakes',
            'shape',
            'shelter',
            'shelters',
            'sheriff',
            'sheriff sale',
            'sheriffs office',
            'shore',
            'shore impact zone',
            'shoreland',
            'shoreland classification',
            'shoreline',
            'short counts',
            'shoulder',
            'sidewalks',
            'sieve',
            'significant existing vegetation stands',
            'significant natural resources',
            'single family residential',
            'sinkhole',
            'sinkhole locations',
  

In [112]:
tag = "ecological" #A tag able to be added to a search inquiry picked from the above JSON

In [113]:
base_url = "https://gisdata.mn.gov/api/3/action/package_search?q=" #Search Inquiry

In [114]:
package_information_url = base_url + tag #Adding together

In [115]:
package_information = requests.get(package_information_url, auth = ('user', 'pass'), verify = False) #Requesting the result
# Printing the result
print(package_information_url)

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\urllib3\connectionpool.py:1020: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gisdata.mn.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


https://gisdata.mn.gov/api/3/action/package_search?q=ecological


In [116]:
# Creating a dictionary
package_dict = json.loads(package_information.content)
# Printing the dictionary
pprint.pprint(package_dict)

{'help': 'https://gisdata.mn.gov/api/3/action/help_show?name=package_search',
 'result': {'count': 52,
            'facets': {},
            'results': [{'author': None,
                         'author_email': None,
                         'creator_user_id': None,
                         'extras': [{'key': 'dsAccessConst', 'value': 'None'},
                                    {'key': 'dsCurrentRef',
                                     'value': 'The ECS polygons were '
                                              'delineated during part of the '
                                              'process of identifying Land '
                                              'Type Associations in 1997-1998 '
                                              'and are a refinement of the '
                                              'original ECS'},
                                    {'key': 'dsMetadataUrl',
                                     'value': 'https://resources.gisdata.mn.gov/pub/gd

                                    {'key': 'dsModifiedDate',
                                     'value': '2021-02-24 00:37:51'},
                                    {'key': 'dsOriginator',
                                     'value': 'Minnesota Department of Natural '
                                              'Resources (DNR)'},
                                    {'key': 'dsPurpose',
                                     'value': 'The data is to help make '
                                              'regional scale land use '
                                              'decisions, especially as it '
                                              'relates to balancing '
                                              'development and natural '
                                              'resource protection.'},
                                    {'key': 'gdrsDsGuid',
                                     'value': '{e4c527ca-9d18-4a0e-81b1-fe81df28348e}'},
                   

                                     'value': '2000 LandSat images'},
                                    {'key': 'dsMetadataUrl',
                                     'value': 'https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dnr/env_dnr_r3_resource_sig_areas/metadata/metadata.html'},
                                    {'key': 'dsModifiedDate',
                                     'value': '2021-02-24 00:16:36'},
                                    {'key': 'dsOriginator',
                                     'value': 'Minnesota Department of Natural '
                                              'Resources (DNR)'},
                                    {'key': 'dsPurpose',
                                     'value': 'The data is to help make '
                                              'regional scale land use '
                                              'decisions, especially as it '
                                              'relates to balancing '
         

                                              'the parent data sets.  The data '
                                              'used to interpret these '
                                              'characteristics were of 1994 to '
                                              '1997 vintage.  One exception is '
                                              'that iron mining activities '
                                              '(e.g. open pits, overburden '
                                              'piles, and tailings basins) '
                                              'have significantly altered the '
                                              'landscape since the digital '
                                              'elevation model data was '
                                              'captured to create shaded '
                                              'relief maps.  As new data '
                                              'becomes available, LTA '


                         'extras': [{'key': 'dsAccessConst', 'value': 'None'},
                                    {'key': 'dsMetadataUrl',
                                     'value': 'https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dnr/env_cropland_and_eco_analysis/metadata/metadata.html'},
                                    {'key': 'dsModifiedDate',
                                     'value': '2021-09-09 00:38:29'},
                                    {'key': 'dsOriginator',
                                     'value': 'Minnesota Department of Natural '
                                              'Resources (DNR)'},
                                    {'key': 'dsPeriodOfContent',
                                     'value': '2/6/2013'},
                                    {'key': 'dsPurpose',
                                     'value': 'The data is to help make '
                                              'regional scale land use '
                       

                                        'id': '7e02c0d9-4049-41e4-ad2a-4148cecdb17a',
                                        'last_modified': None,
                                        'mimetype': None,
                                        'mimetype_inner': None,
                                        'name': 'Static Preview - Sample Image',
                                        'package_id': '2f44afd2-e0e0-49b5-ae69-6fbc107ee88f',
                                        'position': 0,
                                        'resource_type': 'preview',
                                        'revision_id': '2e6b2af1-c2cb-4fe1-862c-adef4da0f474',
                                        'size': None,
                                        'state': 'active',
                                        'url': 'https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dnr/bdry_dnr_eco_and_water_res_admin/metadata/preview.jpg',
                                        'url_type': 

                                       {'cache_last_updated': None,
                                        'cache_url': None,
                                        'created': '2021-04-06T08:03:30.839514',
                                        'description': 'View in Data '
                                                       'Repository for U of MN '
                                                       '(DRUM)',
                                        'format': 'HTML',
                                        'gdrsResGuid': '{a9b4864c-3d1c-4a30-86f0-838a2c2b220e}',
                                        'hash': '',
                                        'id': '9083fba8-0b92-45a0-aafa-1e92d98c9865',
                                        'last_modified': None,
                                        'mimetype': None,
                                        'mimetype_inner': None,
                                        'name': 'Download from Data Repository '
                

In [117]:
# Putting the list of dictionaries that is the results into the variable 'package_dict_result'
package_dict_result = package_dict['result']['results']

In [118]:
# Looping through the first item in the list of dictionaries and printing out the keys
# This shows the metadata
for keys in package_dict_result[0]:
    print(keys)

In [119]:
# Requesting shapefiles in the results
r = requests.get(package_dict_result[0]['resources'][3]["url"])
# Opening the shapefiles and wrting them
open('shp_geos_ecological_class_system.zip', 'wb').write(r.content)
print('extracting the content...')

extracting the content...


In [120]:
# Unzipping the shapefiles to the directory on my computer
with zipfile.ZipFile('shp_geos_ecological_class_system.zip', 'r') as zip_ref:
    zip_ref.extractall('C:/Users/ecava/OneDrive/Documents/ArcGIS/Projects/Lab1/Shapefiles')

## Manipulating the Datasets

After the datasets are downloaded to a directory on your computer, they can be manipulated using the ArcPy module.

In [121]:
# Adding the 'Provinces' shapefile to the geodatabase
arcpy.conversion.FeatureClassToGeodatabase(r"C:\Users\ecava\OneDrive\Documents\ArcGIS\Projects\Lab1\Shapefiles\ecs_provinces_of_mn_v99a.shp", r"C:\Users\ecava\OneDrive\Documents\ArcGIS\Projects\Lab1\Lab_1.gdb")

<Result 'C:\\Users\\ecava\\OneDrive\\Documents\\ArcGIS\\Projects\\Lab1\\Lab_1.gdb'>

In [122]:
# Adding the 'Subsections' shapefile to the geodatabase
arcpy.conversion.FeatureClassToGeodatabase(r"C:\Users\ecava\OneDrive\Documents\ArcGIS\Projects\Lab1\Shapefiles\ecs_subsections_of_mn_v99a.shp", r"C:\Users\ecava\OneDrive\Documents\ArcGIS\Projects\Lab1\Lab_1.gdb")

<Result 'C:\\Users\\ecava\\OneDrive\\Documents\\ArcGIS\\Projects\\Lab1\\Lab_1.gdb'>

In [123]:
# Transforming one dataset to the selected coordinate reference system
arcpy.management.Project("ecs_subsections_of_mn_v99a", r"C:\Users\ecava\OneDrive\Documents\ArcGIS\Projects\Lab1\Lab_1.gdb\ecs_subsections_of_mn_v99a_P", 'PROJCS["NAD_1983_UTM_Zone_15N",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-93.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]]', None, 'PROJCS["NAD_1983_UTM_Zone_15N",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-93.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]]', "NO_PRESERVE_SHAPE", None, "NO_VERTICAL")

<Result 'C:\\Users\\ecava\\OneDrive\\Documents\\ArcGIS\\Projects\\Lab1\\Lab_1.gdb\\ecs_subsections_of_mn_v99a_P'>

In [124]:
# Transforming one dataset to the selected coordinate reference system
arcpy.management.Project("ecs_provinces_of_mn_v99a", r"C:\Users\ecava\OneDrive\Documents\ArcGIS\Projects\Lab1\Lab_1.gdb\ecs_provinces_of_mn_v99a_P", 'PROJCS["NAD_1983_UTM_Zone_15N",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-93.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]]', None, 'PROJCS["NAD_1983_UTM_Zone_15N",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-93.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]]', "NO_PRESERVE_SHAPE", None, "NO_VERTICAL")

<Result 'C:\\Users\\ecava\\OneDrive\\Documents\\ArcGIS\\Projects\\Lab1\\Lab_1.gdb\\ecs_provinces_of_mn_v99a_P'>

In [125]:
# Joining the datasets
# And adding the resulting shapefile to the geodatabase
arcpy.analysis.SpatialJoin("ecs_subsections_of_mn_v99a_P", "ecs_provinces_of_mn_v99a_P", r"C:\Users\ecava\OneDrive\Documents\ArcGIS\Projects\Lab1\Lab_1.gdb\provinces_subsection_joined", "JOIN_ONE_TO_MANY", "KEEP_COMMON", 'AREA "AREA" true true false 8 Double 0 0,First,#,ecs_subsections_of_mn_v99a_P,AREA,-1,-1;PERIMETER "PERIMETER" true true false 8 Double 0 0,First,#,ecs_subsections_of_mn_v99a_P,PERIMETER,-1,-1;ECSLTPY2_ "ECSLTPY2_" true true false 8 Double 0 0,First,#,ecs_subsections_of_mn_v99a_P,ECSLTPY2_,-1,-1;ECSLTPY2_I "ECSLTPY2_I" true true false 8 Double 0 0,First,#,ecs_subsections_of_mn_v99a_P,ECSLTPY2_I,-1,-1;PROVNAME "PROVNAME" true true false 35 Text 0 0,First,#,ecs_subsections_of_mn_v99a_P,PROVNAME,0,35;ECS_PROV "ECS_PROV" true true false 3 Text 0 0,First,#,ecs_subsections_of_mn_v99a_P,ECS_PROV,0,3;SECNAME "SECNAME" true true false 35 Text 0 0,First,#,ecs_subsections_of_mn_v99a_P,SECNAME,0,35;ECS_SEC "ECS_SEC" true true false 4 Text 0 0,First,#,ecs_subsections_of_mn_v99a_P,ECS_SEC,0,4;SUBSECNAME "SUBSECNAME" true true false 40 Text 0 0,First,#,ecs_subsections_of_mn_v99a_P,SUBSECNAME,0,40;ECS_SUBSEC "ECS_SUBSEC" true true false 5 Text 0 0,First,#,ecs_subsections_of_mn_v99a_P,ECS_SUBSEC,0,5;VERSION "VERSION" true true false 5 Text 0 0,First,#,ecs_subsections_of_mn_v99a_P,VERSION,0,5;Shape_Leng "Shape_Leng" true true false 8 Double 0 0,First,#,ecs_subsections_of_mn_v99a_P,Shape_Leng,-1,-1;Shape_Length "Shape_Length" false true true 8 Double 0 0,First,#,ecs_subsections_of_mn_v99a_P,Shape_Length,-1,-1;Shape_Area "Shape_Area" false true true 8 Double 0 0,First,#,ecs_subsections_of_mn_v99a_P,Shape_Area,-1,-1;AREA_1 "AREA" true true false 8 Double 0 0,First,#,ecs_provinces_of_mn_v99a_P,AREA,-1,-1;PERIMETER_1 "PERIMETER" true true false 8 Double 0 0,First,#,ecs_provinces_of_mn_v99a_P,PERIMETER,-1,-1;ECSLTPY21 "ECSLTPY2_" true true false 8 Double 0 0,First,#,ecs_provinces_of_mn_v99a_P,ECSLTPY2_,-1,-1;ECSLTPY2_I_1 "ECSLTPY2_I" true true false 8 Double 0 0,First,#,ecs_provinces_of_mn_v99a_P,ECSLTPY2_I,-1,-1;PROVNAME_1 "PROVNAME" true true false 35 Text 0 0,First,#,ecs_provinces_of_mn_v99a_P,PROVNAME,0,35;ECS_PROV_1 "ECS_PROV" true true false 3 Text 0 0,First,#,ecs_provinces_of_mn_v99a_P,ECS_PROV,0,3;VERSION_1 "VERSION" true true false 5 Text 0 0,First,#,ecs_provinces_of_mn_v99a_P,VERSION,0,5;Shape_Leng_1 "Shape_Leng" true true false 8 Double 0 0,First,#,ecs_provinces_of_mn_v99a_P,Shape_Leng,-1,-1;Shape_Length_1 "Shape_Length" false true true 8 Double 0 0,First,#,ecs_provinces_of_mn_v99a_P,Shape_Length,-1,-1;Shape_Area_1 "Shape_Area" false true true 8 Double 0 0,First,#,ecs_provinces_of_mn_v99a_P,Shape_Area,-1,-1', "INTERSECT", None, '')

<Result 'C:\\Users\\ecava\\OneDrive\\Documents\\ArcGIS\\Projects\\Lab1\\Lab_1.gdb\\provinces_subsection_joined'>

## Checking the Manipulated Datasets

I used arcpy to convert the table of the created shapefile (after join) into an excel file that can be converted to a pandas dataframe to be easily manipulated.

In [126]:
# Converting table of output (joined shapefile) into an excel file
arcpy.conversion.TableToExcel("provinces_subsection_joined", r"C:\Users\ecava\OneDrive\Documents\ArcGIS\Projects\Lab1\joined_table_to_excel.xlsx", "NAME", "CODE")

<Result 'C:\\Users\\ecava\\OneDrive\\Documents\\ArcGIS\\Projects\\Lab1\\joined_table_to_excel.xlsx'>

In [127]:
# Converting the excel file to a pandas dataframe
df = pd.read_excel(r'C:\Users\ecava\OneDrive\Documents\ArcGIS\Projects\Lab1\joined_table_to_excel.xlsx')

In [128]:
# Printing to screen the head of the table showing the merged attributes
df.head()

,OBJECTID,Join_Count,TARGET_FID,JOIN_FID,AREA,PERIMETER,ECSLTPY2_,ECSLTPY2_I,PROVNAME,ECS_PROV,SECNAME,ECS_SEC,SUBSECNAME,ECS_SUBSEC,VERSION,Shape_Leng,AREA_1,PERIMETER_1,ECSLTPY21,ECSLTPY2_I_1,PROVNAME_1,ECS_PROV_1,VERSION_1,Shape_Leng_1,Shape_Length_1,Shape_Area_1,Shape_Length,Shape_Area
0,1,1,1,1,1.478614e+10,1.071015e+06,2,1,Laurentian Mixed Forest Province,212,N. Minnesota & Ontario Peatlands,212M,Agassiz Lowlands,212Mb,2000a,1.071015e+06,9.375321e+10,2.816036e+06,2,1,Laurentian Mixed Forest Province,212,2000a,2.816036e+06,2.816036e+06,9.375321e+10,1.071015e+06,1.478614e+10
1,2,1,1,3,1.478614e+10,1.071015e+06,2,1,Laurentian Mixed Forest Province,212,N. Minnesota & Ontario Peatlands,212M,Agassiz Lowlands,212Mb,2000a,1.071015e+06,1.176094e+10,8.995750e+05,4,3,Tallgrass Aspen Parklands,223,2000a,8.995750e+05,8.995750e+05,1.176094e+10,1.071015e+06,1.478614e+10
2,3,1,2,2,1.598754e+10,1.393452e+06,3,2,Prairie Parkland Province,251,Red River Valley,251A,Red River Prairie,251Aa,2000a,1.393452e+06,6.513418e+10,2.520710e+06,3,2,Prairie Parkland Province,251,2000a,2.520710e+06,2.520710e+06,6.513418e+10,1.393452e+06,1.598754e+10
3,4,1,2,3,1.598754e+10,1.393452e+06,3,2,Prairie Parkland Province,251,Red River Valley,251A,Red River Prairie,251Aa,2000a,1.393452e+06,1.176094e+10,8.995750e+05,4,3,Tallgrass Aspen Parklands,223,2000a,8.995750e+05,8.995750e+05,1.176094e+10,1.393452e+06,1.598754e+10
4,5,1,2,4,1.598754e+10,1.393452e+06,3,2,Prairie Parkland Province,251,Red River Valley,251A,Red River Prairie,251Aa,2000a,1.393452e+06,4.791395e+10,2.788788e+06,5,4,Eastern Broadleaf Forest Province,222,2000a,2.788788e+06,2.788788e+06,4.791395e+10,1.393452e+06,1.598754e+10


## Adding table to Geodatabase

I provided two options for adding the table of the joined shapefile to a geodatabase. Both options utilize arcpy.

In [129]:
# Option 1: Saving the integrated dataset (table of the shapefile) to a geodatabase
arcpy.conversion.TableToGeodatabase("provinces_subsection_joined", r"C:\Users\ecava\OneDrive\Documents\ArcGIS\Projects\Lab1\Lab_1.gdb")

<Result 'C:\\Users\\ecava\\OneDrive\\Documents\\ArcGIS\\Projects\\Lab1\\Lab_1.gdb'>

In [130]:
# Option 2: Import to geodatabase as table
arcpy.conversion.ExcelToTable(r"C:\Users\ecava\OneDrive\Documents\ArcGIS\Projects\Lab1\joined_table_to_excel.xlsx", r"C:\Users\ecava\OneDrive\Documents\ArcGIS\Projects\Lab1\Lab_1.gdb\joined_table_to_excel_ExcelToTable", '', 1, '')

<Result 'C:\\Users\\ecava\\OneDrive\\Documents\\ArcGIS\\Projects\\Lab1\\Lab_1.gdb\\joined_table_to_excel_ExcelToTable'>

## Check

I decided to try out the arcpy.da.Walk function to catalog the data and ensure that the tables were added to the geodatabase.

In [131]:
# Trying out the arcpy walk function to see if the table is located in the folder with the geodatabase
walk = arcpy.da.Walk(r"C:\Users\ecava\OneDrive\Documents\ArcGIS\Projects\Lab1\Lab_1.gdb", datatype = "Table")

In [132]:
# Looping through the directory added to the variable 'walk'
# Adding the info to a blank list 'feature_classes'
feature_classes = []
for dirpath, dirnames, filenames in walk:
    for filename in filenames:
        feature_classes.append(os.path.join(dirpath, filename))

In [133]:
# Printing out the 'feature_classes' list of tables in the geodatabase
pprint.pprint(feature_classes)

['C:\\Users\\ecava\\OneDrive\\Documents\\ArcGIS\\Projects\\Lab1\\Lab_1.gdb\\provinces_subsection_joined_1',
 'C:\\Users\\ecava\\OneDrive\\Documents\\ArcGIS\\Projects\\Lab1\\Lab_1.gdb\\provinces_subsection_joined_2',
 'C:\\Users\\ecava\\OneDrive\\Documents\\ArcGIS\\Projects\\Lab1\\Lab_1.gdb\\joined_table_to_excel_ExcelToTable']
